# Welcome to the Speech Recognition with the Vosk library and Summarization with HuggingFace transformers Project

## 1. Speech recognition using Vosk

In [2]:
from vosk import Model, KaldiRecognizer

In [3]:
Model(model_name="vosk-model-small-en-us-0.15")

In [4]:
FRAME_RATE = 16000
CHANNELS=1

model = Model(model_name="vosk-model-small-en-us-0.15")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

In [5]:
import pydub

In [6]:
from pydub import AudioSegment

In [7]:
from pathlib import Path

import os, sys

print (sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [10]:
AudioSegment.converter = "C:\\PATH_Programs\\ffmpeg.exe"                    
AudioSegment.ffprobe   = "C:\\PATH_Programs\\ffprobe.exe"

audio = AudioSegment.from_mp3("C:\\Users\\ayael\\elonmusk.mp3")
audio = audio.set_channels(CHANNELS)
audio = audio.set_frame_rate(FRAME_RATE)

In [11]:
rec.AcceptWaveform(audio.raw_data)
result = rec.Result()

In [15]:
result

'{\n  "result" : [{\n      "conf" : 0.909778,\n      "end" : 0.584924,\n      "start" : 0.299862,\n      "word" : "the"\n    }, {\n      "conf" : 1.000000,\n      "end" : 0.960000,\n      "start" : 0.763270,\n      "word" : "about"\n    }, {\n      "conf" : 0.584723,\n      "end" : 1.110000,\n      "start" : 0.960000,\n      "word" : "it"\n    }, {\n      "conf" : 0.261871,\n      "end" : 1.410000,\n      "start" : 1.110000,\n      "word" : "aired"\n    }, {\n      "conf" : 1.000000,\n      "end" : 1.530000,\n      "start" : 1.410000,\n      "word" : "for"\n    }, {\n      "conf" : 1.000000,\n      "end" : 1.680000,\n      "start" : 1.535130,\n      "word" : "long"\n    }, {\n      "conf" : 1.000000,\n      "end" : 2.010000,\n      "start" : 1.680000,\n      "word" : "time"\n    }, {\n      "conf" : 1.000000,\n      "end" : 2.250000,\n      "start" : 2.010000,\n      "word" : "since"\n    }, {\n      "conf" : 0.730943,\n      "end" : 2.382700,\n      "start" : 2.250000,\n      "word" :

In [12]:
import json
text = json.loads(result)["text"]

In [13]:
text

"the about it aired for long time since as and cause really is one of the things that this or four five things either would really affect future a dramatically a it it is fundamentally profound in that the this the smartest creatures this was you know on this earth are humans is are defining characteristic yes we're obviously are weaker than say chimpanzees and less agile but are smarter so now what happens when something vastly smarter than the smartest person comes along and silicon form is very difficult to predict what will happen in that circumstance it's called the singularity it's it as a singularity like a black hole it you don't know what happens after that taught to produce"

## Adding punctuation to the transcript with recasepunc

In [18]:
import subprocess

cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)

In [19]:
cased

"The about it aired for long time since as and cause really is one of the things that this or four five things either would really affect future a dramatically a it It is fundamentally profound in that the. This the smartest creatures this was, you know on this earth are humans, is are defining characteristic. Yes, we ' re obviously are weaker than, say, chimpanzees and less agile, but are smarter. So now what happens when something vastly smarter than the smartest person comes along ? And silicon form is very difficult to predict what will happen in that circumstance ? It ' s called the singularity It ' s it as a singularity like a black hole. It you don ' t know what happens after that. Taught to produce.\n"

In [25]:
def voice_recognition(filename):
    model = Model(model_name="vosk-model-en-us-0.22")
    rec = KaldiRecognizer(model, FRAME_RATE)
    rec.SetWords(True)
    
    mp3 = AudioSegment.from_mp3("elonmusk.mp3")
    mp3 = mp3.set_channels(CHANNELS)
    mp3 = mp3.set_frame_rate(FRAME_RATE)
    
    step = 45000
    transcript = ""
    for i in range(0, len(mp3), step):
        print(f"Progress: {i/len(mp3)}")
        segment = mp3[i:i+step]
        rec.AcceptWaveform(segment.raw_data)
        result = rec.Result()
        text = json.loads(result)["text"]
        transcript += text
    
    cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=transcript)
    return cased

In [26]:
transcript = voice_recognition("elonmusk.mp3")

Progress: 0.0
Progress: 0.9410682169894182


In [27]:
print(transcript)

I think about eight AI for a long time since I was in college. Really is one of the things that the sir. Four or five things I thought would really affect the future dramatically. It is fundamentally profound in that the. The the smartest creatures, as far as you know on this earth are humans is a defining characteristic. Yes, we obviously weaker than, say, chimpanzees and less agile, but real smarter. So now what happens when something vastly smarter than the smartest person comes along in Silicon Fall ? It ' s very difficult to predict what will happen in that circumstance. It ' s called the singularity It ' s a singularity like a black hole.You don ' t know what happens after that. It ' s hard.



## 2. Summarizing the transcript using HuggingFace transformers

In [28]:
from transformers import pipeline
summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\ayael\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayael\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [29]:
split_tokens = transcript.split(" ")
docs = []
for i in range(0, len(split_tokens), 850):
    selection = " ".join(split_tokens[i:(i+850)])
    docs.append(selection)

In [30]:
summaries = summarizer(docs)

In [31]:
summary = "\n\n".join([d["summary_text"] for d in summaries])

In [32]:
print(summary)

 The the smartest creatures, as far as you know on this earth are humans is a defining characteristic . So now what happens when something vastly smarter than the smartest person comes along in Silicon Fall ? It 's very difficult to predict what will happen in that circumstance. It ' s called the singularity like a black hole. You don't know what happens after that .
